# Sea Surface Altimetry Data Analysis

For this example we will use gridded sea-surface altimetry data from The Copernicus Marine Environment:

http://marine.copernicus.eu/services-portfolio/access-to-products/?option=com_csw&view=details&product_id=SEALEVEL_GLO_PHY_L4_REP_OBSERVATIONS_008_047

This is a widely used dataset in physical oceanography and climate.

In [1]:
import numpy as np
import xarray as xr
import holoviews as hv
import hvplot.xarray
import hvplot.pandas

/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


ModuleNotFoundError: No module named 'holoviews'

### Open Dataset

Here we load the dataset from the THREDDS service endpoints. Note that this very large dataset initializes nearly instantly, and we can see the full list of variables and coordinates.

In [ ]:
import os
filenames = os.path.join('/g/data/je95/ansr/forcing/ocean/tides/OSU-TPXO9/dist/tides/TPXO9_atlas_nc/', '*.nc')

DS = []
for filename in filenames
    url = 'http://dapds00.nci.org.au/thredds/dodsC/'+filename[]
    data = xr.open_dataset(url)
    DS.append(data)

In [ ]:
DS

### Examine Metadata

For those unfamiliar with this dataset, the variable metadata is very helpful for understanding what the variables actually represent.

In [16]:
for v in ds.data_vars:
    print(v)

con
lon_z
lat_z
hRe
hIm


### Create and Connect to Dask Distributed Cluster

In [ ]:
from dask.distributed import Client, progress
from dask_gateway import Gateway

gateway = Gateway()
cluster = gateway.new_cluster()
cluster.scale(10)
cluster

** ☝️ Don't forget to click the link above to view the scheduler dashboard! **

In [ ]:
client = Client(cluster)
client

## Visually Examine Some of the Data

Let's do a sanity check that the data looks reasonable:

In [ ]:
#ds.hRe.sel(time='1982-08-07', method='nearest').hvplot(colormap='RdBu_r', width=900, height=550, rasterize=True)

## Timeseries of Global Mean Sea Level

Here we make a simple yet fundamental calculation: the rate of increase of global mean sea level over the observational period.

In [18]:
# the number of MB involved in the reduction
ds.hRe.nbytes/1e6

233.3232

In [20]:
# the computationally intensive step
hRe_timeseries = ds.hRe.mean(dim=('nx', 'ny')).load()

In [ ]:
sla_full = sla_timeseries.hvplot(label='full data', grid=True,
                          title='Global Sea Level Rise', 
                          width=800, height=400)
sla_filt = sla_timeseries.rolling(time=365, center=True).mean().hvplot(label='rolling annual mean')
hv.Overlay([sla_full, sla_filt]).options(legend_position='top_left')

In order to understand how the sea level rise is distributed in latitude, we can make a sort of [Hovmöller diagram](https://en.wikipedia.org/wiki/Hovm%C3%B6ller_diagram).

In [ ]:
sla_hov = ds.sla.mean(dim='longitude').load()

In [ ]:
sla_hov.transpose().hvplot(rasterize=True, colormap='RdBu_r', width=900, height=400, clim=(-.2,.2))

We can see that most sea level rise is actually in the Southern Hemisphere.

## Sea Level Variability

We can examine the natural variability in sea level by looking at its standard deviation in time.

In [ ]:
sla_std = ds.sla.std(dim='time').load()
sla_std.name = 'Sea Level Variability [m]'

In [ ]:
sla_std.hvplot(colormap='viridis', width=900, height=550, rasterize=True)